In [1]:
X# ISW Naltural language processing unit
# !pip install nltk
# !pip install sklearn
import pandas as pd
import nltk
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import ast
import string
import  nltk.corpus
nltk.download('stopwords')
from collections import Counter
from nltk.stem import SnowballStemmer
import re
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ivantyshchenko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
ISW = pd.read_csv('ISW.csv', dtype={"main_txt": "O"})
ISW
pure = ISW[['main_text', 'date']]

In [3]:
ISW

,date,title,full_url,main_html,main_html_v2,main_text
0,2022-02-25,Russia-Ukraine Warning Update: Russian Offensi...,https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","[<p align=""center"" style=""text-align: left;""><...",['Russian forces entered major Ukrainian citie...
1,2022-02-26,Russia-Ukraine Warning Update: Russian Offensi...,https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","[<p align=""center"" style=""text-align: left;""><...",['Russian forces’ main axes of advance in the ...
2,2022-02-27,Russia-Ukraine Warning Update: Russian Offensi...,https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","[<p style=""text-align: left;""><strong><span st...","['February 27, 4pm EST', 'The Russian military..."
3,2022-02-28,"Russian Offensive Campaign Assessment, Februar...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","[<p align=""center"" style=""text-align: left;""> ...","['February 28, 3:30pm EST', 'The Russian milit..."
4,2022-03-01,"Russian Offensive Campaign Assessment, March 1",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","[<p align=""center"" style=""text-align: left;""> ...","['March 1, 3:00 pm EST', 'Russian forces are c..."
...,...,...,...,...,...,...
324,2023-01-21,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","[<p align=""center"" style=""text-align: left;""> ...","['January 21, 7:45 pm ET', 'The Ukrainian defe..."
325,2023-01-22,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","[<p align=""center"" style=""text-align: left;""><...","['Kateryna Stepanenko and Frederick W. Kagan',..."
326,2023-01-23,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","[<p align=""center"" style=""text-align: left;""><...","['Karolina Hird, George Barros, Kateryna Stepa..."
327,2023-01-24,"Russian Offensive Campaign Assessment, January...",https://www.understandingwar.org/backgrounder/...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""en"" x...","[<p align=""center"" style=""text-align: left;""> ...","['January 24, 9:30 pm ET', 'A coalition of NAT..."


In [4]:
pure.set_index('date', inplace=True)

In [5]:
def lowercase_text(text):
    sample = ast.literal_eval(text)
    return [i.lower() for i in sample]

pure["main_text"] = pure["main_text"].apply(lowercase_text)
pure2 = pure.copy()
pure2

/var/folders/ks/d6pwjtjd5y7_9mmxkttk6rd00000gn/T/ipykernel_68803/1856688138.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pure["main_text"] = pure["main_text"].apply(lowercase_text)


,main_text
date,
2022-02-25,[russian forces entered major ukrainian cities...
2022-02-26,[russian forces’ main axes of advance in the l...
2022-02-27,"[february 27, 4pm est, the russian military ha..."
2022-02-28,"[february 28, 3:30pm est, the russian military..."
2022-03-01,"[march 1, 3:00 pm est, russian forces are comp..."
...,...
2023-01-21,"[january 21, 7:45 pm et, the ukrainian defense..."
2023-01-22,"[kateryna stepanenko and frederick w. kagan, j..."
2023-01-23,"[karolina hird, george barros, kateryna stepan..."


In [ ]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return [i.translate(str.maketrans('', '', PUNCT_TO_REMOVE)) for i in text]

pure["main_text"] = pure["main_text"].apply(lambda text: remove_punctuation(text))

import re

def clean_text(text_list):
    #list of strings to a single string
    cleaned_text = ' '.join(text_list)

    cleaned_text = re.sub(r'\b(January|February|March|April|May|June|July|August|September|October|November|December)\b', lambda match: match.group(0).lower(), cleaned_text, flags=re.IGNORECASE)

    # remove symbols like '-', '/', '\', '\xa0' etc
    cleaned_text = re.sub(r'[-/\\—\xa0"’“]', ' ', cleaned_text)
    cleaned_text = re.sub(r'[’“]', ' ', cleaned_text)
    cleaned_text = re.sub(r'(march)', ' ', cleaned_text)
    cleaned_text = re.sub(r'\n', ' ', cleaned_text)

    cleaned_text = re.sub(r'\b(?:pm|am)\b', '', cleaned_text)

    cleaned_text = re.sub(r'\b(?:\d+(?:st|nd|rd|th)?)\b', '', cleaned_text)

    cleaned_text = re.sub(r'\b\d+[a-zA-Z]+|(?<!\d)[a-zA-Z]+\d+\b', '', cleaned_text)
    cleaned_text = re.sub(r'\b\w\b', '', cleaned_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text.strip())

    return cleaned_text

pure['new'] = pure['main_text'].apply(lambda x: clean_text(x))


In [ ]:
pure["new"].iloc[13]

In [ ]:
pure["main_text"] = pure["new"]
pure.drop(["new"], axis=1)

In [ ]:
from nltk.corpus import stopwords

", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

pure["text_wo_stop"] = pure["main_text"].apply(lambda text: remove_stopwords(text))
pure.head()


In [ ]:
STOPWORDS

In [ ]:
pure["text_wo_stop"].iloc[13]

In [ ]:
# stemming for comparing with lemtz
stemmer = SnowballStemmer("english")

def stem_words(word_list):
    return [stemmer.stem(word_list)]

pure["text_stemmed"] = pure["text_wo_stop"].apply(stem_words)
pure.iloc[28]["text_stemmed"]

In [ ]:
from nltk.stem import WordNetLemmatizer
# lemmatizing via nltk. another version of lemmatizing lower
lemmatizer = WordNetLemmatizer()
def lemmatize_word(text):
    word_tokens = text.split()
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens]
    return lemmas

pure['text_for_vect'] = pure['text_wo_stop'].apply(lemmatize_word)

text = pure["text_for_vect"].iloc[12]
len(text)

In [ ]:
grouped_pure = pure.groupby('date')['text_for_vect'].first().reset_index()

ISW = pd.merge(ISW, grouped_pure, on='date', how='left')
ISW.to_csv('isw_for_vect.csv', index=False)


In [ ]:
# (pure.iloc[28]["text_for_vect"]
# )

In [ ]:
# ISW.iloc[28]["main_text"]

In [ ]:
ISW["text_for_vect"]

In [ ]:

ISW['text_for_vect_str'] = ISW['text_for_vect'].apply(lambda x: ' '.join(x))

# find rows where 'text_for_vect' contains the word 'affiliated'
rows_with_affiliated = ISW[ISW['text_for_vect_str'].str.contains('affiliated', case=False)]
print("Rows containing the word 'affiliated':")
print(rows_with_affiliated
)


In [ ]:
# print(ISW.iloc[28]["text_for_vect_str"])


In [ ]:
text = ISW.iloc[28]["main_text"]

word = "affiliated"

if word in text:
    index = text.index(word)
    print(f"The word '{word}' is present at index {index}: {text[index:index+len(word)]}")
else:
    print(f"The word '{word}' is not present in the text.")



In [ ]:
text = ISW.iloc[28]["text_for_vect_str"]
word = "affiliated"

if word in text:
    index = text.index(word)
    print(f"The word '{word}' is present at index {index}: {text[index:index+len(word)]}")
else:
    print(f"The word '{word}' is not present in the text.")


# we see that previous lemmatizing doesn't work fully. i'll try another one:

In [ ]:
import spacy
import pandas as pd

# lemmatizing 2
nlp = spacy.load("en_core_web_sm")
def lemmatize_text(word_list):
    text = ' '.join(word_list)
    doc = nlp(text)
    lemmatized_words = [token.lemma_ for token in doc]

    return lemmatized_words

pure['text_for_vect2'] = pure['text_for_vect'].apply(lemmatize_text)

In [ ]:
# pure.iloc[13]['text_for_vect2']

In [ ]:
text = pure.iloc[28]["text_for_vect2"]
word = "affiliated"

if word in text:
    index = text.index(word)
    print(f"The word '{word}' is present at index {index}: {text[index:index+len(word)]}")
else:
    print(f"The word '{word}' is not present in the text.")
# it has to say "is not present"

In [ ]:
# pure['text_for_vect2']

In [ ]:
grouped_pure = pure.groupby('date')['text_for_vect2'].first().reset_index()

ISW = pd.merge(ISW, grouped_pure, on='date', how='left')
ISW.to_csv('isw_vect_upd.csv', index=False)

In [ ]:
new = pd.read_csv('isw_vect_upd.csv')
new

In [ ]:
text = ISW.iloc[28]["text_for_vect_str"]
word = "affiliated"

if word in text:
    index = text.index(word)
    print(f"The word '{word}' is present at index {index}: {text[index:index+len(word)]}")
else:
    print(f"The word '{word}' is not present in the text.")

In [ ]:
ISW.drop(columns=["text_for_vect_str", "text_for_vect", "test", "text_for_vect2_str"], inplace=True)

In [ ]:
new = pd.read_csv('isw_vect_upd.csv')
new

In [ ]:
new.drop(columns=["text_for_vect_str", "text_for_vect", "test"], inplace=True)

In [ ]:
new

In [ ]:
# comparing two lemmatizations in  text_for_vect2 and text_for_vect
new['text_for_vect2_str'] = new['text_for_vect2'].apply(lambda x: ' '.join(x))
text = new.iloc[28]["text_for_vect2_str"]
word = "affiliated"

if word in text:
    index = text.index(word)
    print(f"The word '{word}' is present at index {index}: {text[index:index+len(word)]}")
else:
    print(f"The word '{word}' is not present in the text.")

In [ ]:
ISW['text_for_vect2_str'] = ISW['text_for_vect'].apply(lambda x: ' '.join(x))
text = ISW.iloc[28]["text_for_vect2_str"]
word = "affiliated"

if word in text:
    index = text.index(word)
    print(f"The word '{word}' is present at index {index}: {text[index:index+len(word)]}")
else:
    print(f"The word '{word}' is not present in the text.")